In [ ]:
import pandas as pd
import numpy as np



# Importation et structuration des données

In [ ]:
df = pd.read_csv('/content/donnees.csv', header=None)

df.columns = ['prix', 'vitesse_max', 'consommation', 'freinage',
              'confort', 'volume_coffre', 'acceleration']

noms_voitures = [
    'Alfa_156', 'Audi_A4', 'Cit_Xantia', 'Peugeot_406', 'Saab_TID',
    'Rnlt_Laguna', 'VW_Passat', 'BMW_320d', 'Cit_Xsara', 'Rnlt_Safrane'
]
df.insert(0, 'modele', noms_voitures)


# Normalisation des données

In [ ]:
def normaliser_donnees(df):
    """
    Applique une normalisation min-max classique sur les colonnes spécifiées (0 à 1).
    :param df: DataFrame original
    :return: DataFrame avec les colonnes normalisées
    """
    df_norm = df.copy()

    for critere in df.columns:
        # On n'applique pas la normalisation sur la colonne modele
        if critere == 'modele':
            continue

        min_val = df_norm[critere].min()
        max_val = df_norm[critere].max()
        if max_val - min_val == 0:
            df_norm[critere] = 0  # Évite la division par zéro
        else:
            df_norm[critere] = (df_norm[critere] - min_val) / (max_val - min_val)

    return df_norm

In [ ]:
normalised_df = normaliser_donnees(df)
normalised_df

,modele,prix,vitesse_max,consommation,freinage,confort,volume_coffre,acceleration
0,Alfa_156,0.469730,0.703704,0.884615,0.838710,0.714286,0.000000,0.081081
1,Audi_A4,0.663656,0.481481,0.000000,0.225806,0.857143,0.436620,0.567568
2,Cit_Xantia,0.636364,0.481481,0.846154,0.419355,0.142857,0.718310,0.837838
3,Peugeot_406,0.651548,0.333333,1.000000,0.000000,0.142857,0.366197,1.000000
4,Saab_TID,0.704545,0.629630,0.807692,0.209677,0.571429,0.816901,0.297297
5,Rnlt_Laguna,0.454545,0.444444,0.769231,0.483871,0.428571,0.521127,0.783784
6,VW_Passat,0.422787,0.481481,0.730769,0.000000,0.285714,0.683099,0.729730
7,BMW_320d,0.712187,1.000000,0.346154,0.354839,0.428571,0.436620,0.000000
8,Cit_Xsara,0.000000,0.000000,0.269231,1.000000,1.000000,0.211268,0.702703
9,Rnlt_Safrane,1.000000,0.777778,0.692308,0.016129,0.000000,1.000000,0.297297


# Somme pondérée

In [ ]:
def somme_pondere(df, colonnes_criteres, poids, a_minimiser=None):
    """
    Calcule les scores avec pondération en gérant les critères à minimiser.

    :param df: DataFrame avec les données
    :param colonnes_criteres: liste des colonnes à utiliser
    :param poids: liste des poids (même ordre que colonnes_criteres)
    :param a_minimiser: liste optionnelle des colonnes à minimiser
    """
    if len(poids) != len(colonnes_criteres):
        raise ValueError("Le nombre de poids doit correspondre au nombre de critères.")

    df_SP = df.copy()

    # Inversion des critères à minimiser
    if a_minimiser:
        for critere in a_minimiser:
            if critere in colonnes_criteres:
                df_SP[critere] = df_SP[critere].max() - df_SP[critere]

    # Calcul du score pondéré
    df_SP['score'] = df_SP[colonnes_criteres].mul(poids).sum(axis=1)

    # Meilleur modèle
    meilleur = df_SP.loc[df_SP['score'].idxmax()]
    print(f"Le meilleur modèle : {meilleur['modele']} avec un score de {meilleur['score']:.2f}")

    return df_SP.sort_values(by='score', ascending=False)


## Données non normalisées vs données normalisées

**Poids égaux**

In [ ]:
columns = ['prix', 'vitesse_max', 'consommation', 'freinage',
              'confort', 'volume_coffre', 'acceleration']

poids = [0.1] * len(columns)
somme_pondere(df, columns, poids, a_minimiser = None)
somme_pondere(normalised_df, columns, poids, a_minimiser = None)

Le meilleur modèle : Rnlt_Safrane avec un score de 2995.80
Le meilleur modèle : Cit_Xantia avec un score de 0.41


,modele,prix,vitesse_max,consommation,freinage,confort,volume_coffre,acceleration,score
2,Cit_Xantia,0.636364,0.481481,0.846154,0.419355,0.142857,0.718310,0.837838,0.408236
4,Saab_TID,0.704545,0.629630,0.807692,0.209677,0.571429,0.816901,0.297297,0.403717
5,Rnlt_Laguna,0.454545,0.444444,0.769231,0.483871,0.428571,0.521127,0.783784,0.388557
9,Rnlt_Safrane,1.000000,0.777778,0.692308,0.016129,0.000000,1.000000,0.297297,0.378351
0,Alfa_156,0.469730,0.703704,0.884615,0.838710,0.714286,0.000000,0.081081,0.369213
3,Peugeot_406,0.651548,0.333333,1.000000,0.000000,0.142857,0.366197,1.000000,0.349394
6,VW_Passat,0.422787,0.481481,0.730769,0.000000,0.285714,0.683099,0.729730,0.333358
7,BMW_320d,0.712187,1.000000,0.346154,0.354839,0.428571,0.436620,0.000000,0.327837
1,Audi_A4,0.663656,0.481481,0.000000,0.225806,0.857143,0.436620,0.567568,0.323227
8,Cit_Xsara,0.000000,0.000000,0.269231,1.000000,1.000000,0.211268,0.702703,0.318320


In [ ]:
columns = ['prix', 'vitesse_max', 'consommation', 'freinage',
              'confort', 'volume_coffre', 'acceleration']

poids = [0.1] * len(columns)
somme_pondere(df, columns, poids, a_minimiser = ['prix', 'consommation'])
somme_pondere(normalised_df, columns, poids, a_minimiser = ['prix', 'consommation'])

Le meilleur modèle : Cit_Xsara avec un score de 1075.00
Le meilleur modèle : Cit_Xsara avec un score de 0.46


,modele,prix,vitesse_max,consommation,freinage,confort,volume_coffre,acceleration,score
8,Cit_Xsara,1.000000,0.000000,0.730769,1.000000,1.000000,0.211268,0.702703,0.464474
1,Audi_A4,0.336344,0.481481,1.000000,0.225806,0.857143,0.436620,0.567568,0.390496
5,Rnlt_Laguna,0.545455,0.444444,0.230769,0.483871,0.428571,0.521127,0.783784,0.343802
7,BMW_320d,0.287813,1.000000,0.653846,0.354839,0.428571,0.436620,0.000000,0.316169
2,Cit_Xantia,0.363636,0.481481,0.153846,0.419355,0.142857,0.718310,0.837838,0.311732
6,VW_Passat,0.577213,0.481481,0.269231,0.000000,0.285714,0.683099,0.729730,0.302647
4,Saab_TID,0.295455,0.629630,0.192308,0.209677,0.571429,0.816901,0.297297,0.301270
0,Alfa_156,0.530270,0.703704,0.115385,0.838710,0.714286,0.000000,0.081081,0.298343
9,Rnlt_Safrane,0.000000,0.777778,0.307692,0.016129,0.000000,1.000000,0.297297,0.239890
3,Peugeot_406,0.348452,0.333333,0.000000,0.000000,0.142857,0.366197,1.000000,0.219084


**Poids différents**

In [ ]:
poids_perso1 = [0.3, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1]

somme_pondere(normalised_df, columns, poids_perso1, a_minimiser = None)


Le meilleur modèle : Rnlt_Safrane avec un score de 0.66


,modele,prix,vitesse_max,consommation,freinage,confort,volume_coffre,acceleration,score
9,Rnlt_Safrane,1.000000,0.777778,0.692308,0.016129,0.000000,1.000000,0.297297,0.656129
4,Saab_TID,0.704545,0.629630,0.807692,0.209677,0.571429,0.816901,0.297297,0.607589
2,Cit_Xantia,0.636364,0.481481,0.846154,0.419355,0.142857,0.718310,0.837838,0.583657
7,BMW_320d,0.712187,1.000000,0.346154,0.354839,0.428571,0.436620,0.000000,0.570275
0,Alfa_156,0.469730,0.703704,0.884615,0.838710,0.714286,0.000000,0.081081,0.533529
5,Rnlt_Laguna,0.454545,0.444444,0.769231,0.483871,0.428571,0.521127,0.783784,0.523911
3,Peugeot_406,0.651548,0.333333,1.000000,0.000000,0.142857,0.366197,1.000000,0.513037
1,Audi_A4,0.663656,0.481481,0.000000,0.225806,0.857143,0.436620,0.567568,0.504107
6,VW_Passat,0.422787,0.481481,0.730769,0.000000,0.285714,0.683099,0.729730,0.466064
8,Cit_Xsara,0.000000,0.000000,0.269231,1.000000,1.000000,0.211268,0.702703,0.318320


In [ ]:
poids_perso1 = [0.1, 0.5, 0.1 , 0.1, 0.1, 0.5, 0.1]
somme_pondere(df, columns, poids_perso1, a_minimiser = ['prix', 'consommation'])
somme_pondere(normalised_df, columns, poids_perso1, a_minimiser = ['prix', 'consommation'])

Le meilleur modèle : Cit_Xsara avec un score de 1311.00
Le meilleur modèle : Rnlt_Safrane avec un score de 0.95


,modele,prix,vitesse_max,consommation,freinage,confort,volume_coffre,acceleration,score
9,Rnlt_Safrane,0.000000,0.777778,0.307692,0.016129,0.000000,1.000000,0.297297,0.951001
7,BMW_320d,0.287813,1.000000,0.653846,0.354839,0.428571,0.436620,0.000000,0.890817
4,Saab_TID,0.295455,0.629630,0.192308,0.209677,0.571429,0.816901,0.297297,0.879882
2,Cit_Xantia,0.363636,0.481481,0.153846,0.419355,0.142857,0.718310,0.837838,0.791649
6,VW_Passat,0.577213,0.481481,0.269231,0.000000,0.285714,0.683099,0.729730,0.768479
1,Audi_A4,0.336344,0.481481,1.000000,0.225806,0.857143,0.436620,0.567568,0.757737
5,Rnlt_Laguna,0.545455,0.444444,0.230769,0.483871,0.428571,0.521127,0.783784,0.730031
0,Alfa_156,0.530270,0.703704,0.115385,0.838710,0.714286,0.000000,0.081081,0.579825
8,Cit_Xsara,1.000000,0.000000,0.730769,1.000000,1.000000,0.211268,0.702703,0.548981
3,Peugeot_406,0.348452,0.333333,0.000000,0.000000,0.142857,0.366197,1.000000,0.498896


In [ ]:
poids_perso2 = [0.3, 0.1, 0.1 , 0.2, 0.1, 0.1, 0.1]

somme_pondere(df, columns, poids_perso2, a_minimiser = ['prix', 'consommation'])
somme_pondere(normalised_df, columns, poids_perso2, a_minimiser = ['prix', 'consommation'])

Le meilleur modèle : Cit_Xsara avec un score de 3094.26
Le meilleur modèle : Cit_Xsara avec un score de 0.76


,modele,prix,vitesse_max,consommation,freinage,confort,volume_coffre,acceleration,score
8,Cit_Xsara,1.000000,0.000000,0.730769,1.000000,1.000000,0.211268,0.702703,0.764474
5,Rnlt_Laguna,0.545455,0.444444,0.230769,0.483871,0.428571,0.521127,0.783784,0.501280
0,Alfa_156,0.530270,0.703704,0.115385,0.838710,0.714286,0.000000,0.081081,0.488268
1,Audi_A4,0.336344,0.481481,1.000000,0.225806,0.857143,0.436620,0.567568,0.480346
2,Cit_Xantia,0.363636,0.481481,0.153846,0.419355,0.142857,0.718310,0.837838,0.426395
6,VW_Passat,0.577213,0.481481,0.269231,0.000000,0.285714,0.683099,0.729730,0.418089
7,BMW_320d,0.287813,1.000000,0.653846,0.354839,0.428571,0.436620,0.000000,0.409215
4,Saab_TID,0.295455,0.629630,0.192308,0.209677,0.571429,0.816901,0.297297,0.381328
3,Peugeot_406,0.348452,0.333333,0.000000,0.000000,0.142857,0.366197,1.000000,0.288774
9,Rnlt_Safrane,0.000000,0.777778,0.307692,0.016129,0.000000,1.000000,0.297297,0.241503




## Promethee I et II

In [ ]:
import numpy as np
import pandas as pd

def preference_binaire(diff):
    """
    Fonction de préférence binaire :
    Si la différence est positive (ai > aj sur le critère), alors 1, sinon 0
    """
    return 1 if diff > 0 else 0

def promethee_version_binaire(df, colonnes_criteres, poids, a_minimiser=None):
    """
    PROMETHEE I/II version stricte du cours :
    - fonction de préférence binaire
    - on ajoute directement le poids si ai > aj sur un critère
    """
    n = len(df)
    preference_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            preference_score = 0
            for idx, critere in enumerate(colonnes_criteres):
                val_i = df.loc[i, critere]
                val_j = df.loc[j, critere]

                # Inversion pour les critères à minimiser
                if a_minimiser and critere in a_minimiser:
                    diff = val_j - val_i
                else:
                    diff = val_i - val_j

                p = preference_binaire(diff)  # 1 ou 0
                preference_score += poids[idx] * p  # poids ajouté directement

            preference_matrix[i, j] = preference_score

    # Flux sortant et entrant
    phi_plus = preference_matrix.sum(axis=1) / (n - 1)
    phi_minus = preference_matrix.sum(axis=0) / (n - 1)
    phi = phi_plus - phi_minus

    resultats = pd.DataFrame({
        'modele': df['modele'],
        'Φ+': phi_plus,
        'Φ-': phi_minus,
        'flux_net (Φ+ - Φ-)': phi
    })
    return resultats


In [ ]:
poids = [0.3, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1]
promethee_version_binaire(normalised_df, columns, poids, a_minimiser=['prix', 'consommation'])

,modele,Φ+,Φ-,flux_net (Φ+ - Φ-)
0,Alfa_156,0.544444,0.455556,0.088889
1,Audi_A4,0.477778,0.466667,0.011111
2,Cit_Xantia,0.500000,0.444444,0.055556
3,Peugeot_406,0.288889,0.688889,-0.400000
4,Saab_TID,0.444444,0.544444,-0.100000
5,Rnlt_Laguna,0.577778,0.411111,0.166667
6,VW_Passat,0.555556,0.388889,0.166667
7,BMW_320d,0.444444,0.533333,-0.088889
8,Cit_Xsara,0.655556,0.344444,0.311111
9,Rnlt_Safrane,0.388889,0.600000,-0.211111


In [ ]:
def afficher_classements_promethee(resultats):
    """
    Affiche les classements PROMETHEE I (flux Φ+ et Φ−) et PROMETHEE II (flux net).
    :param resultats: DataFrame contenant 'modele', 'Φ+', 'Φ-', 'flux_net (Φ+ - Φ-)'.
    """

    print("\n🔷 PROMETHEE I - Classement selon Φ+ (flux sortant, à maximiser) :")
    classement_phi_plus = resultats.sort_values(by='Φ+', ascending=False).reset_index(drop=True)
    for i, row in classement_phi_plus.iterrows():
        print(f"{i+1}. {row['modele']} - Φ+ = {row['Φ+']:.4f}")

    print("\n🔷 PROMETHEE I - Classement selon Φ- (flux entrant, à minimiser) :")
    classement_phi_minus = resultats.sort_values(by='Φ-', ascending=True).reset_index(drop=True)
    for i, row in classement_phi_minus.iterrows():
        print(f"{i+1}. {row['modele']} - Φ- = {row['Φ-']:.4f}")

    print("\n✅ PROMETHEE II - Classement selon flux net (Φ+ - Φ−) :")
    classement_phi_net = resultats.sort_values(by='flux_net (Φ+ - Φ-)', ascending=False).reset_index(drop=True)
    for i, row in classement_phi_net.iterrows():
        print(f"{i+1}. {row['modele']} - Φ = {row['flux_net (Φ+ - Φ-)']:.4f}")


In [ ]:
resultat =promethee_version_binaire(normalised_df, columns, poids, a_minimiser=['prix', 'consommation'])
afficher_classements_promethee(resultat)


🔷 PROMETHEE I - Classement selon Φ+ (flux sortant, à maximiser) :
1. Cit_Xsara - Φ+ = 0.6556
2. Rnlt_Laguna - Φ+ = 0.5778
3. VW_Passat - Φ+ = 0.5556
4. Alfa_156 - Φ+ = 0.5444
5. Cit_Xantia - Φ+ = 0.5000
6. Audi_A4 - Φ+ = 0.4778
7. BMW_320d - Φ+ = 0.4444
8. Saab_TID - Φ+ = 0.4444
9. Rnlt_Safrane - Φ+ = 0.3889
10. Peugeot_406 - Φ+ = 0.2889

🔷 PROMETHEE I - Classement selon Φ- (flux entrant, à minimiser) :
1. Cit_Xsara - Φ- = 0.3444
2. VW_Passat - Φ- = 0.3889
3. Rnlt_Laguna - Φ- = 0.4111
4. Cit_Xantia - Φ- = 0.4444
5. Alfa_156 - Φ- = 0.4556
6. Audi_A4 - Φ- = 0.4667
7. BMW_320d - Φ- = 0.5333
8. Saab_TID - Φ- = 0.5444
9. Rnlt_Safrane - Φ- = 0.6000
10. Peugeot_406 - Φ- = 0.6889

✅ PROMETHEE II - Classement selon flux net (Φ+ - Φ−) :
1. Cit_Xsara - Φ = 0.3111
2. Rnlt_Laguna - Φ = 0.1667
3. VW_Passat - Φ = 0.1667
4. Alfa_156 - Φ = 0.0889
5. Cit_Xantia - Φ = 0.0556
6. Audi_A4 - Φ = 0.0111
7. BMW_320d - Φ = -0.0889
8. Saab_TID - Φ = -0.1000
9. Rnlt_Safrane - Φ = -0.2111
10. Peugeot_406 - Φ = -0

In [ ]:
def preference_non_binaire(diff):
    """Fonction de préférence simple (si diff > 0 alors préférence = diff, sinon 0)."""
    return max(0, diff)

def promethee(df, colonnes_criteres, poids, a_minimiser=None):
    n = len(df)
    preference_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            preference_score = 0
            for idx, critere in enumerate(colonnes_criteres):
                val_i = df.loc[i, critere]
                val_j = df.loc[j, critere]

                # Inversion si critère à minimiser
                if a_minimiser and critere in a_minimiser:
                    diff = val_j - val_i
                else:
                    diff = val_i - val_j

                p = preference_non_binaire(diff)
                preference_score += poids[idx] * p
            preference_matrix[i, j] = preference_score

    # Flux sortants et entrants
    phi_plus = preference_matrix.sum(axis=1) / (n - 1)
    phi_minus = preference_matrix.sum(axis=0) / (n - 1)
    phi = phi_plus - phi_minus

    resultats = pd.DataFrame({
        'modele': df['modele'],
        'Φ+': phi_plus,
        'Φ-': phi_minus,
        'flux_net (Φ+ - Φ-)': phi
    })

    return resultats


In [ ]:
resultat =promethee(normalised_df, columns, poids, a_minimiser=['prix', 'consommation'])
afficher_classements_promethee(resultat)



🔷 PROMETHEE I - Classement selon Φ+ (flux sortant, à maximiser) :
1. Cit_Xsara - Φ+ = 0.3937
2. Alfa_156 - Φ+ = 0.1888
3. BMW_320d - Φ+ = 0.1856
4. Audi_A4 - Φ+ = 0.1789
5. Rnlt_Laguna - Φ+ = 0.1535
6. VW_Passat - Φ+ = 0.1428
7. Rnlt_Safrane - Φ+ = 0.1286
8. Cit_Xantia - Φ+ = 0.1183
9. Saab_TID - Φ+ = 0.1180
10. Peugeot_406 - Φ+ = 0.0828

🔷 PROMETHEE I - Classement selon Φ- (flux entrant, à minimiser) :
1. Rnlt_Laguna - Φ- = 0.1095
2. Audi_A4 - Φ- = 0.1255
3. VW_Passat - Φ- = 0.1334
4. Cit_Xantia - Φ- = 0.1463
5. Saab_TID - Φ- = 0.1563
6. Cit_Xsara - Φ- = 0.1641
7. BMW_320d - Φ- = 0.1679
8. Alfa_156 - Φ- = 0.1700
9. Peugeot_406 - Φ- = 0.2336
10. Rnlt_Safrane - Φ- = 0.2843

✅ PROMETHEE II - Classement selon flux net (Φ+ - Φ−) :
1. Cit_Xsara - Φ = 0.2296
2. Audi_A4 - Φ = 0.0534
3. Rnlt_Laguna - Φ = 0.0439
4. Alfa_156 - Φ = 0.0188
5. BMW_320d - Φ = 0.0177
6. VW_Passat - Φ = 0.0094
7. Cit_Xantia - Φ = -0.0280
8. Saab_TID - Φ = -0.0383
9. Peugeot_406 - Φ = -0.1508
10. Rnlt_Safrane - Φ = -0

# ELECTRE I

In [ ]:
# Fonction ELECTRE I
def calcul_concordance(df_norm, poids, seuil_concordance):
    n = len(df_norm)
    concordance_matrix = np.zeros((n, n))

    criteres_a_minimiser = ['prix', 'consommation']

    for i in range(n):
        for j in range(n):
            if i == j:
                continue

            somme_poids = 0
            for critere in poids.keys():
                val_i = df_norm.loc[i, critere]
                val_j = df_norm.loc[j, critere]

                if critere in criteres_a_minimiser:
                    if val_i <= val_j:
                        somme_poids += poids[critere]
                else:
                    if val_i >= val_j:
                        somme_poids += poids[critere]

            concordance_matrix[i, j] = somme_poids

    surclassement = concordance_matrix >= seuil_concordance
    return concordance_matrix, surclassement

# Exemple de poids (somme = 1)
poids_criteres = {
    'prix': 0.3,
    'vitesse_max': 0.2,
    'consommation': 0.1,
    'freinage': 0.1,
    'confort': 0.1,
    'volume_coffre': 0.1,
    'acceleration': 0.1
}

# Seuil de concordance (entre 0.5 et 0.7 typiquement)
seuil = 0.6

# Application de ELECTRE I
concordance, relation_surclassement = calcul_concordance(normalised_df, poids_criteres, seuil)

# Affichage sous forme de DataFrames
voitures = df['modele'].tolist()
df_concordance = pd.DataFrame(concordance, index=voitures, columns=voitures)
df_surclassement = pd.DataFrame(relation_surclassement.astype(int), index=voitures, columns=voitures)

print("Matrice de concordance :")
display(df_concordance)

print("\nMatrice de surclassement (1 = surclassement, 0 = non) :")
display(df_surclassement)

Matrice de concordance :


,Alfa_156,Audi_A4,Cit_Xantia,Peugeot_406,Saab_TID,Rnlt_Laguna,VW_Passat,BMW_320d,Cit_Xsara,Rnlt_Safrane
Alfa_156,0.0,0.6,0.7,0.8,0.7,0.4,0.4,0.6,0.2,0.5
Audi_A4,0.4,0.0,0.4,0.6,0.7,0.4,0.5,0.7,0.4,0.7
Cit_Xantia,0.3,0.8,0.0,0.9,0.5,0.4,0.5,0.6,0.4,0.6
Peugeot_406,0.2,0.4,0.2,0.0,0.4,0.1,0.2,0.4,0.4,0.5
Saab_TID,0.3,0.3,0.5,0.6,0.0,0.4,0.5,0.6,0.3,0.6
Rnlt_Laguna,0.6,0.6,0.6,0.9,0.6,0.0,0.3,0.7,0.4,0.6
VW_Passat,0.6,0.7,0.7,0.9,0.5,0.7,0.0,0.5,0.4,0.5
BMW_320d,0.4,0.4,0.4,0.6,0.4,0.4,0.5,0.0,0.3,0.8
Cit_Xsara,0.8,0.6,0.6,0.6,0.7,0.6,0.6,0.7,0.0,0.7
Rnlt_Safrane,0.5,0.3,0.4,0.5,0.5,0.4,0.5,0.2,0.3,0.0



Matrice de surclassement (1 = surclassement, 0 = non) :


,Alfa_156,Audi_A4,Cit_Xantia,Peugeot_406,Saab_TID,Rnlt_Laguna,VW_Passat,BMW_320d,Cit_Xsara,Rnlt_Safrane
Alfa_156,0,1,1,1,1,0,0,1,0,0
Audi_A4,0,0,0,1,1,0,0,1,0,1
Cit_Xantia,0,1,0,1,0,0,0,1,0,1
Peugeot_406,0,0,0,0,0,0,0,0,0,0
Saab_TID,0,0,0,1,0,0,0,1,0,1
Rnlt_Laguna,1,1,1,1,1,0,0,1,0,1
VW_Passat,1,1,1,1,0,1,0,0,0,0
BMW_320d,0,0,0,1,0,0,0,0,0,1
Cit_Xsara,1,1,1,1,1,1,1,1,0,1
Rnlt_Safrane,0,0,0,0,0,0,0,0,0,0


In [ ]:
def afficher_relations_surclassement(df_surclassement):
    relations = []
    lignes = df_surclassement.index.tolist()
    for i in range(len(lignes)):
        for j in range(len(lignes)):
            if i != j and df_surclassement.iloc[i, j] == 1:
                relations.append(f"{lignes[i]} surclasse {lignes[j]}")
    return relations

relations = afficher_relations_surclassement(df_surclassement)

# Affichage clair
print("Relations de surclassement identifiées :\n")
for r in relations:
    print("- " + r)

Relations de surclassement identifiées :

- Alfa_156 surclasse Audi_A4
- Alfa_156 surclasse Cit_Xantia
- Alfa_156 surclasse Peugeot_406
- Alfa_156 surclasse Saab_TID
- Alfa_156 surclasse BMW_320d
- Audi_A4 surclasse Peugeot_406
- Audi_A4 surclasse Saab_TID
- Audi_A4 surclasse BMW_320d
- Audi_A4 surclasse Rnlt_Safrane
- Cit_Xantia surclasse Audi_A4
- Cit_Xantia surclasse Peugeot_406
- Cit_Xantia surclasse BMW_320d
- Cit_Xantia surclasse Rnlt_Safrane
- Saab_TID surclasse Peugeot_406
- Saab_TID surclasse BMW_320d
- Saab_TID surclasse Rnlt_Safrane
- Rnlt_Laguna surclasse Alfa_156
- Rnlt_Laguna surclasse Audi_A4
- Rnlt_Laguna surclasse Cit_Xantia
- Rnlt_Laguna surclasse Peugeot_406
- Rnlt_Laguna surclasse Saab_TID
- Rnlt_Laguna surclasse BMW_320d
- Rnlt_Laguna surclasse Rnlt_Safrane
- VW_Passat surclasse Alfa_156
- VW_Passat surclasse Audi_A4
- VW_Passat surclasse Cit_Xantia
- VW_Passat surclasse Peugeot_406
- VW_Passat surclasse Rnlt_Laguna
- BMW_320d surclasse Peugeot_406
- BMW_320d surcl

# Electre Is

In [ ]:
def electre_is(df, colonnes_criteres, poids, seuil_concordance=0.65, seuil_discordance=0.4):
    """
    Implémentation d’ELECTRE IS.

    :param df: DataFrame
    :param colonnes_criteres: Liste des colonnes utilisées comme critères
    :param poids: Liste des poids pour chaque critère
    :param seuil_concordance: Seuil minimal pour valider le surclassement
    :param seuil_discordance: Seuil maximal autorisé pour la discordance
    :return: liste des alternatives non dominées et matrices C, D, S
    """
    df = df.reset_index(drop=True)
    n = len(df)
    C = np.zeros((n, n))  # Concordance
    D = np.zeros((n, n))  # Discordance
    S = np.zeros((n, n))  # Surclassement

    for i in range(n):
        for j in range(n):
            if i == j:
                continue

            concordance = 0
            discordances = []

            for k, critere in enumerate(colonnes_criteres):
                val_i = df.loc[i, critere]
                val_j = df.loc[j, critere]

                # Concordance : poids si a_i >= a_j
                if val_i >= val_j:
                    concordance += poids[k]

                # Discordance partielle (si a_j > a_i)
                diff = val_j - val_i
                discordances.append(diff)

            C[i, j] = concordance
            D[i, j] = max(discordances)

            if C[i, j] >= seuil_concordance and D[i, j] <= seuil_discordance:
                S[i, j] = 1  # a_i surclasse a_j

    # Alternatives non dominées
    dominé = set(np.where(S.T == 1)[0])
    toutes = set(range(n))
    non_dominées = list(toutes - dominé)

    return non_dominées, C, D, S


## Electre Is sans normalisation

In [ ]:
colonnes_criteres = ['prix', 'vitesse_max', 'consommation', 'freinage',
                     'confort', 'volume_coffre', 'acceleration']
poids = [0.3, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1]

# ELECTRE IS
non_dominées, C, D, S = electre_is(df, colonnes_criteres, poids)

# Affichage
print("Classement selon ELECTRE IS :")
for i in non_dominées:
    print("-", df.loc[i, 'modele'])

Classement selon ELECTRE IS :
- Alfa_156
- Audi_A4
- Cit_Xantia
- Peugeot_406
- Saab_TID
- Rnlt_Laguna
- VW_Passat
- BMW_320d
- Cit_Xsara
- Rnlt_Safrane


## Electre Is avec normalisation

In [ ]:
colonnes_criteres = ['prix', 'vitesse_max', 'consommation', 'freinage',
                     'confort', 'volume_coffre', 'acceleration']
poids = [0.3, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1]

df_norm = normaliser_donnees(df)
# ELECTRE IS
non_dominées, C, D, S = electre_is(df_norm, colonnes_criteres, poids)

# Affichage
print("Classement selon ELECTRE IS :")
for i in non_dominées:
    print("-", df.loc[i, 'modele'])

Classement selon ELECTRE IS :
- Alfa_156
- Cit_Xantia
- Peugeot_406
- Saab_TID
- BMW_320d
- Cit_Xsara
- Rnlt_Safrane
